In [1]:
import os
import pandas as pd 
from glob import glob

In [2]:
files = glob("data/github-dataset/warehouse/*.csv")
dfs = [pd.read_csv(file) for file in files]

In [3]:
df = dfs[3]

In [4]:
df.head(1)

,url,repository_url,id,user,node_id,title,state,labels,created_at,updated_at,body
0,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas,2428583885,sfc-gh-joshi,I_kwDOAA0YD86QwT_N,BUG: GroupBy.value_counts doesn't preserve ori...,open,Bug,2024-07-24T22:41:33Z,2024-07-24T22:41:33Z,### Pandas version checks\n\n- [X] I have chec...


In [6]:
import torch
from tqdm import tqdm
from mlengine.core.model import train_mlm
from transformers import AutoModelForMaskedLM, AutoTokenizer

/home/yikuang/miniconda3/envs/rapids-defectsearch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
entry_corpus = df.body

file_path = f'{os.path.abspath("")}/train.txt'

with open(file_path, 'w') as f:
    for line in entry_corpus:
        f.write(f"{line}\n")

# https://www.sbert.net/docs/sentence_transformer/pretrained_models.html
train_mlm("sentence-transformers/all-mpnet-base-v2", file_path)

In [7]:
output_path = 'output/sentence-transformers_all-mpnet-base-v2-2024-07-28_17-15-23'
saved_model_path = f'{os.path.abspath("")}/{output_path}/'

tokenizer = AutoTokenizer.from_pretrained(saved_model_path)
model = AutoModelForMaskedLM.from_pretrained(saved_model_path, output_hidden_states=True)

In [8]:
import chromadb
chroma_client = chromadb.PersistentClient(path = "./data/chromadb/")

In [10]:
[i.name for i in chroma_client.list_collections()]

['gh2024', 'cleaned-gh2024']

In [34]:
try: collection = chroma_client.get_collection(name ="gh2024")
except: collection = chroma_client.create_collection(name="gh2024")

In [21]:
df = df.dropna(subset = ['body'])

In [13]:
import random 

corpus_embeddings, i = [], 0

len(corpus)

11684

#### Generate Text Embeddings

In [27]:
df.reset_index(drop = True, inplace = True)

In [35]:
import uuid

chunk_size = 1_000

for row in tqdm(df.itertuples(), total = df.shape[0]):

    encoded_input = tokenizer(
        row.body, 
        padding=True, 
        truncation=True, 
        max_length=128, 
        return_tensors='pt'
    )
    
    with torch.no_grad():
        output = model(**encoded_input)
        embeddings = output.hidden_states[-1].mean(dim=1)  # Using mean pooling
        
    corpus_embeddings.append(embeddings[0].tolist())

    # Insert the embedded sentences into the database on every chunk_size chunk
    if i % chunk_size == 0 and i != 0:
        
        collection.upsert(
            documents=df.iloc[i - chunk_size: i]['body'].to_list(),
            embeddings=corpus_embeddings[i - chunk_size: i],
            ids=df.iloc[i - chunk_size: i]["node_id"].to_list(),
        )
        
    i += 1

100%|██████████████████████████████████████████████████████████████████████████████████| 11563/11563 [13:04<00:00, 14.74it/s]
